## Use feature-weighted rf model on the crcns vim-1 dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import pickle
from time import time
from glob import glob
from scipy.io import loadmat
from PIL import Image
from os.path import join
from scipy.stats import pearsonr
from hrf_fitting.src.feature_weighted_rf_models import make_rf_table,receptive_fields, model_space, prediction_menu, bigmult
from hrf_fitting.src.feature_weighted_rf_models import train_fwrf_model
from hrf_fitting.src.gabor_feature_dictionaries import gabor_feature_maps

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
version_number = '1p1'

### Step 0: Load crcns stimuli

#### load crcns stimuli 

In [ ]:
##known stimulus parameters
Ttrn = 1750
Tval = 120
S = 500
T = Ttrn+Tval
train_stim_files = glob('/media/tnaselar/Data/crcns_datasets/vim-1/Stimuli_Trn_FullRes*.mat')
val_stim_file = '/media/tnaselar/Data/crcns_datasets/vim-1/Stimuli_Val_FullRes.mat'
n_image_channels = 1 ##could be 3 for color images.

In [ ]:
##allocate memory for stim
training_stim = np.zeros((Ttrn,S,S),dtype='float32')

##load training stim
cnt = 0
for sl in sorted(train_stim_files):
    this_h5 = h5py.File(sl,'r')
    this_train_stim = this_h5['stimTrn']
    this_num_stim = this_train_stim.shape[-1]
    training_stim[cnt:cnt+this_num_stim,:,:] = np.transpose(this_train_stim[:],[2,1,0])
    cnt += this_num_stim
    this_h5.close()
    
##load validation stim
val_h5 = h5py.File(val_stim_file,'r')
validation_stim = np.transpose(val_h5['stimVal'][:],[2,1,0])
val_h5.close()

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(validation_stim[0,:,:],cmap='gray')
plt.subplot(1,2,2)
plt.imshow(training_stim[-1,:,:],cmap = 'gray')

### Step 1: construct feature maps

In [ ]:
n_orientations = 8
deg_per_stimulus = 20
lowest_sp_freq = .25 ##cyc/deg
highest_sp_freq = 6.25
num_sp_freq = 8
pix_per_cycle = 4#2.13333333
complex_cell = True

print 'D = total number of features = %d' %(n_orientations * num_sp_freq)

#### construct gabor wavelet stack

In [ ]:
gfm = gabor_feature_maps(n_orientations,
                         deg_per_stimulus,
                         (lowest_sp_freq,highest_sp_freq,num_sp_freq),
                         pix_per_cycle=pix_per_cycle,complex_cell=complex_cell,
                         diams_per_filter = 4,
                        cycles_per_radius = 2.0)

In [ ]:
gfm.gbr_table.head(9)

In [ ]:
gfm.filter_stack.shape

#### see one of the gabors

In [ ]:
o =  8##choose an orientation
plt.imshow(np.imag(gfm.filter_stack[o,0,:,:]),cmap='gray')

### Step 2: receptive fields

In [ ]:
deg_per_radius = (.25, 8, 8) ##rf sizes in degrees (smallest, largest, number of sizes)
spacing = .75 ##spacing between rf's in degrees
rf = receptive_fields(deg_per_stimulus,deg_per_radius,spacing)

In [ ]:
rf.rf_table.head()

In [ ]:
print 'G = number of rf models = %d' %(rf.rf_table.shape[0])

### Step 3: Model space

#### apply an activation function

In [ ]:
def log_act_func(x):
    return np.log(1+np.sqrt(x))

#### instantiate model space object

In [ ]:
##construct the model space
init_feat_dict = gfm.create_feature_maps(training_stim[0,np.newaxis,np.newaxis,:,:])
ms = model_space(init_feat_dict, rf, activation_function = log_act_func)

#### construct training/validation model space tensors

In [ ]:
##loop over training stimuli because feature maps for all training stim. > 48Gb
training_mst = np.zeros((ms.receptive_fields.G, Ttrn, ms.D)).astype('float32')

num_chunks = 2
stim_dx = np.linspace(0,T-1,num=num_chunks+1, endpoint=True,dtype='int')

cnt = 0
for t in stim_dx[1:]:
    this_training_feature_dict = gfm.create_feature_maps(training_stim[cnt:cnt+t,np.newaxis,:,:])
    training_mst[:,cnt:cnt+t,:] = ms.construct_model_space_tensor(this_training_feature_dict,normalize=False)
    cnt += t

##clear up memory
this_training_feature_dict = []

##normalize and save normalization constants
training_mst = ms.normalize_model_space_tensor(training_mst, save=True)


##should work in one shot for because not too big
val_feature_dict = gfm.create_feature_maps(validation_stim[:,np.newaxis,:,:])

In [ ]:
validation_mst = ms.construct_model_space_tensor(val_feature_dict,normalize=True)

### Step 4: load data and fit models in a loop

In [ ]:
def save_stuff(save_to_this_file, data_objects_dict):
    failed = []
    with h5py.File(save_to_this_file+'.h5py', 'w') as hf:
        for k,v in data_objects_dict.iteritems():
            try:
                hf.create_dataset(k,data=v)
                print 'saved %s in h5py file' %(k)
            except:
                failed.append(k)
                print 'failed to save %s as h5py. will try pickle' %(k)
    
    for k in failed:
        with open(save_to_this_file+'_'+'%s.pkl' %(k), 'w') as pkl:
            try:
                pickle.dump(data_objects_dict[k],pkl)
                print 'saved %s as pkl' %(k)
            except:
                print 'failed to save %s in any format. lost.' %(k)
    
    

In [ ]:
subjects = ['S1','S2']
saving_dir = '/media/tnaselar/Data/fwrf/Gabor_model/'
voxel_file = '/media/tnaselar/Data/crcns_datasets/vim-1/EstimatedResponses.mat'


In [ ]:
with  h5py.File(voxel_file,'r') as crcns_voxel_data:
    for subj in subjects:
        saving_file = join(saving_dir, 'gabor_model_%s' %(subj))
        save_this_data = {}
        save_this_data['model_space'] = ms
        #remove nans, becuase this data-set has some. otherwise even one nan will infect gradient for every voxel.
        voxel_data = np.concatenate((crcns_voxel_data['dataVal'+subj],crcns_voxel_data['dataTrn'+subj])
                                    ,axis=0).astype('float32')
        no_nan = np.isnan(voxel_data).sum(axis=0) == 0 ##<<pull voxels with nans 
        voxel_data = voxel_data[:,no_nan]
        print voxel_data.shape
        V = voxel_data.shape[1]
        #get training/validation views on voxel_data
        nvox=V
        trnIdx = np.arange(Tval,T)
        valIdx = np.arange(0,Tval)
        trn_voxel_data = voxel_data[trnIdx,0:nvox]
        val_voxel_data = voxel_data[valIdx,0:nvox]
        #train the model!
        fvl,ffw,frf,beh = train_fwrf_model(training_mst,
                                           trn_voxel_data,
                                           initial_feature_weights='zeros',
                                           voxel_binsize = nvox,
                                           rf_grid_binsize=10,
                                           learning_rate=10**(-5.3),
                                           max_iters = 200,
                                           early_stop_fraction=0.2,
                                           report_every = 100)
                                           
                                          

        save_this_data['fvl'] = fvl
        save_this_data['ffw'] = ffw
        save_this_data['frf'] = frf
        save_this_data['beh'] = beh

        ##generate predictions one voxel at a time
        pred = np.zeros((Tval,nvox))
        for v in range(nvox):  ##<<some kind of bug in training function, last voxel getting skipped.
            pred[:,v] = np.squeeze(bigmult(validation_mst[np.newaxis,frf[v],:,:],
                                           ffw[np.newaxis,:,v, np.newaxis]))
        save_this_data['pred'] = pred

        ##get correlation = prediction accuracy
        val_cc = np.zeros(nvox)  
        for v in range(nvox): 
            val_cc[v] = pearsonr(val_voxel_data[:,v],pred[:,v])[0]
        val_cc = np.nan_to_num(val_cc)
        save_this_data['val_cc'] = val_cc

        save_stuff(saving_file, {})

In [ ]:
##close data file
crcns_voxel_data.close()